In [53]:
import numpy as np
import pandas as pd
df_24 = pd.read_excel('data.xlsx', sheet_name='2024')
df_23 = pd.read_excel('data.xlsx', sheet_name='2023')
df_22 = pd.read_excel('data.xlsx', sheet_name='2022')
df_21 = pd.read_excel('data.xlsx', sheet_name='2021')
df_20 = pd.read_excel('data.xlsx', sheet_name='2020')
mkc = pd.read_excel('Vietnam_Marketcap.xlsx')
price = pd.read_excel('Vietnam_Price.xlsx')
volume = pd.read_excel('Vietnam_volume.xlsx', sheet_name = 'Sheet2')
print(df_24.head())
print(mkc.head())
print(price.head())
print(volume.head())

    Mã   Sàn Ngành ICB - cấp 2     Năm  \
0  ACB  HOSE         Ngân hàng  2024.0   
1  BCM  HOSE      Bất động sản  2024.0   
2  BID  HOSE         Ngân hàng  2024.0   
3  BVH  HOSE          Bảo hiểm  2024.0   
4  CTG  HOSE         Ngân hàng  2024.0   

   Lợi nhuận sau thuế thu nhập doanh nghiệp  TỔNG CỘNG TÀI SẢN  \
0                              16789.768000       8.640057e+05   
1                               2309.721094       5.877743e+04   
2                              25121.739000       2.760693e+06   
3                               2157.033005       2.512178e+05   
4                              25475.009000       2.385384e+06   

   GTCL tài sản cố định vô hình   NỢ PHẢI TRẢ  VỐN CHỦ SỞ HỮU  
0                   2190.172000  7.805440e+05    83461.678000  
1                    128.539543  3.829753e+04    20479.901455  
2                   5320.612000  2.616182e+06   144511.549000  
3                   1018.134513  2.277112e+05    23506.572713  
4                   3852.94100

In [54]:
def merge_market_data(df, price_df, mkc_df, volume_df, target_date):
    """
    Hàm merge dữ liệu tài chính với giá, vốn hóa và khối lượng giao dịch
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame chứa dữ liệu tài chính chính (ví dụ: df_24, df_23...)
    price_df : pandas.DataFrame
        DataFrame chứa dữ liệu giá cổ phiếu
    mkc_df : pandas.DataFrame
        DataFrame chứa dữ liệu vốn hóa thị trường
    volume_df : pandas.DataFrame
        DataFrame chứa dữ liệu khối lượng giao dịch
    target_date : str
        Ngày mục tiêu để lấy dữ liệu (format: "YYYY-MM-DD")
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame đã được merge với các dữ liệu thị trường
    """
    # Tạo copy để không thay đổi dữ liệu gốc
    price = price_df.copy()
    mkc = mkc_df.copy()
    volume = volume_df.copy()
    
    # --- Chuẩn hóa cột Code ---
    price["Mã"] = price["Code"].str.extract(r"VT:(\w+)\(P\)")
    mkc["Mã"] = mkc["Code"].str.extract(r"VT:(\w+)\(MV\)")
    volume["Mã"] = volume["Code"].str.extract(r"VT:(\w+)\(VO\)")
    
    # --- Chuẩn hóa cột ngày về dạng datetime ---
    price.columns = [pd.to_datetime(c, errors="ignore") if c not in ["Name", "Code", "Mã"] else c for c in price.columns]
    mkc.columns = [pd.to_datetime(c, errors="ignore") if c not in ["Name", "Code", "Mã"] else c for c in mkc.columns]
    volume.columns = [pd.to_datetime(c, errors="ignore") if c not in ["Name", "Code", "Mã"] else c for c in volume.columns]
    
    # --- Chọn ngày mục tiêu ---
    target_datetime = pd.to_datetime(target_date)
    
    # --- Tìm ngày gần nhất có dữ liệu ---
    def find_nearest_date(df, target_date):
        """Tìm ngày gần nhất <= target_date có dữ liệu"""
        date_cols = [c for c in df.columns if isinstance(c, pd.Timestamp) and c <= target_date]
        if date_cols:
            return max(date_cols)  # Lấy ngày gần nhất
        else:
            return None
    
    # Tìm ngày thực tế sẽ sử dụng cho price và mkc
    actual_price_date = find_nearest_date(price, target_datetime)
    actual_mkc_date = find_nearest_date(mkc, target_datetime)
    
    # --- Lấy giá và vốn hóa tại ngày thực tế ---
    if actual_price_date is not None:
        price_day = price[["Mã", actual_price_date]].rename(columns={actual_price_date: "price"})
        print(f"Sử dụng dữ liệu giá từ ngày: {actual_price_date.strftime('%Y-%m-%d')}")
    else:
        print(f"Cảnh báo: Không có dữ liệu giá nào <= {target_date}")
        price_day = pd.DataFrame(columns=["Mã", "price"])
    
    if actual_mkc_date is not None:
        mkc_day = mkc[["Mã", actual_mkc_date]].rename(columns={actual_mkc_date: "marketcap"})
        print(f"Sử dụng dữ liệu vốn hóa từ ngày: {actual_mkc_date.strftime('%Y-%m-%d')}")
    else:
        print(f"Cảnh báo: Không có dữ liệu vốn hóa nào <= {target_date}")
        mkc_day = pd.DataFrame(columns=["Mã", "marketcap"])
    
    # --- Tính avg20_volume (20 ngày gần nhất <= target_date) ---
    vol_date_cols = [c for c in volume.columns if isinstance(c, pd.Timestamp) and c <= target_datetime]
    vol_date_cols = sorted(vol_date_cols)[-20:]  # lấy 20 cột gần nhất
    
    if vol_date_cols:
        vol_avg20 = (
            volume[["Mã"] + vol_date_cols]
            .set_index("Mã")
            .apply(pd.to_numeric, errors="coerce")
            .mean(axis=1, skipna=True)
            .rename("avg20_volume")
            .reset_index()
        )
        print(f"Tính avg20_volume từ {len(vol_date_cols)} ngày gần nhất, ngày cuối: {max(vol_date_cols).strftime('%Y-%m-%d')}")
    else:
        print(f"Cảnh báo: Không có dữ liệu khối lượng cho ngày <= {target_date}")
        vol_avg20 = pd.DataFrame(columns=["Mã", "avg20_volume"])
    
    # --- Ghép vào df ---
    df_result = df.copy()
    df_result = df_result.merge(price_day, on="Mã", how="left")
    df_result = df_result.merge(mkc_day, on="Mã", how="left")
    df_result = df_result.merge(vol_avg20, on="Mã", how="left")
    
    return df_result

In [55]:
def calculate_financial_ratios(df_merge):
    """
    Hàm tính toán các chỉ số tài chính
    
    Parameters:
    -----------
    df_merge : pandas.DataFrame
        DataFrame đã được merge với dữ liệu thị trường (có các cột: marketcap, price, 
        "Lợi nhuận sau thuế thu nhập doanh nghiệp", "TỔNG CỘNG TÀI SẢN", 
        "GTCL tài sản cố định vô hình", "NỢ PHẢI TRẢ", "VỐN CHỦ SỞ HỮU")
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame với các chỉ số tài chính đã được tính (Slcp, eps, pe, bvps, pb, roe)
    """
    df_result = df_merge.copy()
    
    # Số lượng cổ phiếu lưu hành (Share Outstanding)
    df_result["Slcp"] = df_result["marketcap"] * 1_000_000 / df_result["price"]
    
    # Thu nhập trên mỗi cổ phiếu (Earnings Per Share)
    df_result["eps"] = df_result["Lợi nhuận sau thuế thu nhập doanh nghiệp"] * 1_000_000_000 / df_result["Slcp"]
    
    # Tỷ số giá/thu nhập (Price to Earnings ratio)
    df_result["pe"] = df_result["price"] / df_result["eps"]
    
    # Giá trị sổ sách trên mỗi cổ phiếu (Book Value Per Share)
    df_result["bvps"] = (
        df_result["TỔNG CỘNG TÀI SẢN"] - 
        df_result["GTCL tài sản cố định vô hình"] - 
        df_result["NỢ PHẢI TRẢ"]
    ) * 1_000_000_000 / df_result["Slcp"]
    
    # Tỷ số giá/giá trị sổ sách (Price to Book ratio)
    df_result["pb"] = df_result["price"] / df_result["bvps"]
    
    # Tỷ suất sinh lời trên vốn chủ sở hữu (Return on Equity)
    df_result["roe"] = df_result["Lợi nhuận sau thuế thu nhập doanh nghiệp"] / df_result["VỐN CHỦ SỞ HỮU"]
    
    return df_result


In [56]:
# Sử dụng hàm cho năm 2024
df_merge24 = merge_market_data(df_24, price, mkc, volume, "2024-12-31")
df_final24 = calculate_financial_ratios(df_merge24)
print("Dữ liệu 2024:")
print(df_final24.head())

Sử dụng dữ liệu giá từ ngày: 2024-12-31
Sử dụng dữ liệu vốn hóa từ ngày: 2024-12-31
Tính avg20_volume từ 20 ngày gần nhất, ngày cuối: 2024-12-31
Dữ liệu 2024:
    Mã   Sàn Ngành ICB - cấp 2     Năm  \
0  ACB  HOSE         Ngân hàng  2024.0   
1  BCM  HOSE      Bất động sản  2024.0   
2  BID  HOSE         Ngân hàng  2024.0   
3  BVH  HOSE          Bảo hiểm  2024.0   
4  CTG  HOSE         Ngân hàng  2024.0   

   Lợi nhuận sau thuế thu nhập doanh nghiệp  TỔNG CỘNG TÀI SẢN  \
0                              16789.768000       8.640057e+05   
1                               2309.721094       5.877743e+04   
2                              25121.739000       2.760693e+06   
3                               2157.033005       2.512178e+05   
4                              25475.009000       2.385384e+06   

   GTCL tài sản cố định vô hình   NỢ PHẢI TRẢ  VỐN CHỦ SỞ HỮU    price  \
0                   2190.172000  7.805440e+05    83461.678000  25800.0   
1                    128.539543  3.829753e+

In [57]:
df_merge23 = merge_market_data(df_23, price, mkc, volume, "2023-12-31")
df_final23 = calculate_financial_ratios(df_merge23)
print("\nDữ liệu 2023:")
print(df_final23.head())

Sử dụng dữ liệu giá từ ngày: 2023-12-29
Sử dụng dữ liệu vốn hóa từ ngày: 2023-12-29
Tính avg20_volume từ 20 ngày gần nhất, ngày cuối: 2023-12-29

Dữ liệu 2023:
    Mã   Sàn Ngành ICB - cấp 2     Năm  \
0  ACB  HOSE         Ngân hàng  2023.0   
1  BCM  HOSE      Bất động sản  2023.0   
2  BID  HOSE         Ngân hàng  2023.0   
3  BVH  HOSE          Bảo hiểm  2023.0   
4  CTG  HOSE         Ngân hàng  2023.0   

   Lợi nhuận sau thuế thu nhập doanh nghiệp  TỔNG CỘNG TÀI SẢN  \
0                              16044.733000       7.187946e+05   
1                               2280.087767       5.342390e+04   
2                              21977.141000       2.300869e+06   
3                               1859.989244       2.211016e+05   
4                              20044.622000       2.032614e+06   

   GTCL tài sản cố định vô hình   NỢ PHẢI TRẢ  VỐN CHỦ SỞ HỮU     price  \
0                   1444.993000  6.478386e+05    70955.961000  20782.61   
1                    140.713644  3.39510

In [58]:
df_merge22 = merge_market_data(df_22, price, mkc, volume, "2022-12-31")
df_final22 = calculate_financial_ratios(df_merge22)
print("\nDữ liệu 2022:")
print(df_final22.head())

Sử dụng dữ liệu giá từ ngày: 2022-12-30
Sử dụng dữ liệu vốn hóa từ ngày: 2022-12-30
Tính avg20_volume từ 20 ngày gần nhất, ngày cuối: 2022-12-30

Dữ liệu 2022:
    Mã   Sàn Ngành ICB - cấp 2     Năm  \
0  ACB  HOSE         Ngân hàng  2022.0   
1  BCM  HOSE      Bất động sản  2022.0   
2  BID  HOSE         Ngân hàng  2022.0   
3  BVH  HOSE          Bảo hiểm  2022.0   
4  CTG  HOSE         Ngân hàng  2022.0   

   Lợi nhuận sau thuế thu nhập doanh nghiệp  TỔNG CỘNG TÀI SẢN  \
0                              13688.193000       6.078752e+05   
1                               1714.339267       4.828958e+04   
2                              18420.014000       2.120609e+06   
3                               1625.606380       2.016640e+05   
4                              16834.994000       1.808430e+06   

   GTCL tài sản cố định vô hình   NỢ PHẢI TRẢ  VỐN CHỦ SỞ HỮU     price  \
0                   1131.644000  5.494365e+05    58438.663000  16559.55   
1                    153.307495  3.03444

In [59]:
df_merge21 = merge_market_data(df_21, price, mkc, volume, "2021-12-31")
df_final21 = calculate_financial_ratios(df_merge21)
print("\nDữ liệu 2021:")
print(df_final21.head())

Sử dụng dữ liệu giá từ ngày: 2021-12-31
Sử dụng dữ liệu vốn hóa từ ngày: 2021-12-31
Tính avg20_volume từ 20 ngày gần nhất, ngày cuối: 2021-12-31

Dữ liệu 2021:
    Mã   Sàn Ngành ICB - cấp 2     Năm  \
0  ACB  HOSE         Ngân hàng  2021.0   
1  BCM  HOSE      Bất động sản  2021.0   
2  BID  HOSE         Ngân hàng  2021.0   
3  BVH  HOSE          Bảo hiểm  2021.0   
4  CTG  HOSE         Ngân hàng  2021.0   

   Lợi nhuận sau thuế thu nhập doanh nghiệp  TỔNG CỘNG TÀI SẢN  \
0                               9602.746000       5.277699e+05   
1                               1503.686614       4.895244e+04   
2                              10841.271000       1.761696e+06   
3                               2018.414008       1.695046e+05   
4                              14215.342000       1.531587e+06   

   GTCL tài sản cố định vô hình   NỢ PHẢI TRẢ  VỐN CHỦ SỞ HỮU     price  \
0                   1077.478000  4.828690e+05    44900.909000  20869.58   
1                    149.270158  3.18075

In [60]:
df_merge20 = merge_market_data(df_20, price, mkc, volume, "2020-12-31")
df_final20 = calculate_financial_ratios(df_merge20)
print("\nDữ liệu 2020:")
print(df_final20.head())

Sử dụng dữ liệu giá từ ngày: 2020-12-31
Sử dụng dữ liệu vốn hóa từ ngày: 2020-12-31
Tính avg20_volume từ 20 ngày gần nhất, ngày cuối: 2020-12-31

Dữ liệu 2020:
    Mã   Sàn Ngành ICB - cấp 2     Năm  \
0  ACB  HOSE         Ngân hàng  2020.0   
1  BCM  HOSE      Bất động sản  2020.0   
2  BID  HOSE         Ngân hàng  2020.0   
3  BVH  HOSE          Bảo hiểm  2020.0   
4  CTG  HOSE         Ngân hàng  2020.0   

   Lợi nhuận sau thuế thu nhập doanh nghiệp  TỔNG CỘNG TÀI SẢN  \
0                               7682.823000       4.445301e+05   
1                               2186.010795       4.848533e+04   
2                               7223.565000       1.516686e+06   
3                               1649.846973       1.464128e+05   
4                              13757.234000       1.341436e+06   

   GTCL tài sản cố định vô hình   NỢ PHẢI TRẢ  VỐN CHỦ SỞ HỮU     price  \
0                   1065.838000  4.090819e+05    35448.163000  13598.50   
1                    134.856235  3.12978

In [62]:
df_final24.to_csv("2024.csv", index=False, encoding="utf-8-sig")
df_final23.to_csv("2023.csv", index=False, encoding="utf-8-sig")
df_final22.to_csv("2022.csv", index=False, encoding="utf-8-sig")
df_final21.to_csv("2021.csv", index=False, encoding="utf-8-sig")
df_final20.to_csv("2020.csv", index=False, encoding="utf-8-sig")